In [268]:
import numpy as np
import math
from random import  choice, randint, seed, random
from functools import reduce
from itertools import product
from copy import copy
from scipy import sparse

In [269]:
def fitness2(state,problem_size):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(problem_size)]),
        )
    )
    return valid, -cost

In [270]:
def tweak(state,problem_size):
    new_state = copy(state)
    index = randint(0, problem_size - 1)
    new_state[index] = not new_state[index]
    return new_state

In [271]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [272]:
def simulated_annealing(initial_state, size_of_problem):
    current_state = initial_state
    cooling_rate=0.97
    cost=fitness2(current_state,size_of_problem)
    best_cost=cost
    temperature=cooling_rate
    last_cost=fitness2(current_state,size_of_problem)
    counter=1
    while temperature > 0.001 :  
        new_state = tweak(current_state,size_of_problem)
        score_new=fitness2(new_state,size_of_problem)
        score_curr=fitness2(current_state,size_of_problem)
        counter+=2
        if  score_new>score_curr or random() < math.exp( -((2*score_curr[0]+score_curr[1])-(2*score_new[0]+score_new[1]))/ temperature):
            current_state = new_state
            if(score_new[0]==size_of_problem):
                    last_cost=score_new
            if score_new>best_cost:
                best_cost = score_new
        
        temperature *= cooling_rate
    if(best_cost<=last_cost):
        best_cost=last_cost
    return best_cost, counter

In [273]:
def tabu_search(initial_state, size_of_problem):
    current_state = initial_state
    best_value = fitness2(current_state, size_of_problem)
    tabu_list = []
    counter = 0

    for _ in range(100):
        neighbors = [tweak(current_state, size_of_problem) for _ in range(100)]

        for neighbor in neighbors:
            neighbor_value = fitness2(neighbor, size_of_problem)
            counter += 1

            if neighbor not in tabu_list and neighbor_value > best_value:
                current_state = neighbor
                best_value = neighbor_value

        tabu_list.append(current_state)
        if len(tabu_list) > 100:
            tabu_list.pop(0)

    return best_value, counter

In [274]:
def iterated_local_search(current_state, size_of_problem):
    steady_state_count = 0
    steady_state_limit = 10
    counter = 0
    for iterations in range(10000):
        new_state = tweak(current_state, size_of_problem)
        if fitness2(new_state, size_of_problem) >= fitness2(current_state, size_of_problem):
            steady_state_count = 0
            counter += 2
            current_state = new_state
        else:
            steady_state_count += 1
            if steady_state_count == steady_state_limit:
                temp_state = [choice([True, False]) for _ in range(size_of_problem)]
                if fitness2(temp_state, size_of_problem) > fitness2(current_state, size_of_problem):
                    current_state = temp_state
                steady_state_count = 0
    return fitness2(current_state, size_of_problem), counter

In [275]:

current_state = [choice([False, False, False, False ]) for _ in range(j)]
SETS=make_set_covering_problem(100,100,0.3)
best_score,n_time =iterated_local_search(current_state,100)

print("best score: ",best_score,"\ncalls :",n_time,"\n")

best score:  (100, -8) 
calls : 28 

